In [ ]:
from bs4 import BeautifulSoup
import requests as req
from tqdm import tqdm 
import time
import pandas as pd
import random

In [ ]:
print("hello world")

In [ ]:
BASE_URL = "https://podcastaddict.com/"

In [ ]:
#load in a large dataframe with names of podcasts to download 
IN_PATH = "/shared/3/projects/benlitterer/podcastData/podNames/spotifySnowball50k.csv"
df = pd.read_csv(IN_PATH, index_col="Unnamed: 0")
df = df.sort_values("total_episodes", ascending=False)

In [127]:
def getPod(searchText): 
    #reformat the search text and make request 
    searchText = searchText.replace(" ", "+")
    searchResponse = req.get(BASE_URL + "?q=" + searchText).text
    
    print(BASE_URL + "?q=" + searchText)
    #make soup from hits 
    hitsSoup = BeautifulSoup(searchResponse)
    
    #TODO: take care of case when we get no hits or not the right one
    
    #grab the first hit
    hitsSoup = hitsSoup.find_all("a", class_="clickeableItemRow")
    links = [hit.get("href") for hit in hitsSoup]

    #only proceed if we got some hits 
    if len(links) > 0: 
        #go to the website for the first hit
        fpSite = req.get(links[0]).text
        fpSite = BeautifulSoup(fpSite)
        fpHead = fpSite.head

        #get name from website page
        fpName = fpSite.find("div", class_="caption", itemprop="name").get_text()

        #TODO: make sure that the first hit has a very similar title to what we searched 
        #after normalizing the text 

        #we can't be positive that this will always be the type of the link 
        fpRss = fpHead.find("link", type="application/rss+xml").get("href")

        #get the xml for the rss feed 
        rssText = req.get(fpRss).text

        #pare the xml 
        rssSoup = BeautifulSoup(rssText, features="xml")

        #get all of the episode links from the xml 
        epLinks = [enclosure.get("url") for enclosure in rssSoup.find_all("enclosure")]

        return fpName, epLinks
    else: 
        return None, []

In [128]:
#load in a large dataframe with names of podcasts to download 
IN_PATH = "/shared/3/projects/benlitterer/podcastData/podNames/spotifySnowball50k.csv"
df = pd.read_csv(IN_PATH, index_col="Unnamed: 0")
df = df.sort_values("total_episodes", ascending=False)

In [129]:
searchText =  "THE HUGE SHOW"
searchText = searchText.replace(" ", "+")

searchResponse = req.get("https://podcastaddict.com/?q=THE+HUGE+SHOW")

#make soup from hits 
hitsSoup = BeautifulSoup(searchResponse.text)

In [130]:
searchResponse.status_code

404

In [94]:
searchResponse

'<!DOCTYPE html>\n<html lang="en">\n<link rel="icon" type="image/png" href="/res/images/favicon-32x32.png" />\n<meta charset="UTF-8">\n<html lang="en">\n<link rel="icon" type="image/png" href="/res/images/favicon-32x32.png" />\n<meta http-equiv="content-type" content="text/html; charset=utf-8" />\n<head>\n<title>Ooops!</title>\n<meta name="viewport" content="width=device-width, initial-scale=1.0, maximum-scale=1.0" />\n<link rel="stylesheet" href="https://stackpath.bootstrapcdn.com/bootstrap/4.5.0/css/bootstrap.min.css" integrity="sha384-9aIt2nRpC12Uk9gS9baDl411NQApFmC26EwAOH8WgZl5MYYxFfc+NcPb1dKGj7Sk" crossorigin="anonymous">\n<script src="https://code.jquery.com/jquery-3.5.1.slim.min.js" integrity="sha384-DfXdz2htPH0lsSSs5nCTpuj/zy4C+OGpamoFVy38MVBnE+IbbVYUew+OrCXaRkfj" crossorigin="anonymous"></script>\n<script src="https://stackpath.bootstrapcdn.com/bootstrap/4.5.0/js/bootstrap.min.js" integrity="sha384-OgVRvuATP1z7JjHLkuOU7Xw704+h835Lr+6QL9UvYjZE3Ipu6Tp75j7Bh/kR0JKI" crossorigin="

In [106]:
searchPhrases = list(df["name"].head(3))

podRows = []

for search in tqdm(searchPhrases): 
    sResult, links = getPod(search)
    podLinks[show] = links
    
    currRow = [[search, sResult, link] for link in links]
    
    podRows = podRows + currRow
    
    #in case of rate limit? 
    #sleep for random time between 0-1 seconds 
    time.sleep(random.random())

  0%|          | 0/3 [00:00<?, ?it/s]

https://podcastaddict.com/?q=THE+HUGE+SHOW


 33%|███▎      | 1/3 [00:00<00:01,  1.04it/s]

https://podcastaddict.com/?q=Best+of+The+Steve+Harvey+Morning+Show


 67%|██████▋   | 2/3 [00:01<00:00,  1.10it/s]

https://podcastaddict.com/?q=The+Ben+Maller+Show


100%|██████████| 3/3 [00:02<00:00,  1.20it/s]


In [100]:
linkDf = pd.DataFrame(podRows, columns=["search", "result", "link"])

In [101]:
linkDf.shape

(0, 3)

In [102]:
linkDf[linkDf["search"] != linkDf["result"]]

,search,result,link


In [114]:
#what about the itunes search api  
IURL = "https://itunes.apple.com/search?" 
response = req.get(IURL + "term=Joe+Rogan&media=podcast")

In [115]:
response.text

'\n\n\n{\n "resultCount":10,\n "results": [\n{"wrapperType":"track", "kind":"podcast", "collectionId":1396757201, "trackId":1396757201, "artistName":"Joe Rogan Experience Review podcast", "collectionName":"Joe Rogan Experience Review podcast", "trackName":"Joe Rogan Experience Review podcast", "collectionCensoredName":"Joe Rogan Experience Review podcast", "trackCensoredName":"Joe Rogan Experience Review podcast", "collectionViewUrl":"https://podcasts.apple.com/us/podcast/joe-rogan-experience-review-podcast/id1396757201?uo=4", "feedUrl":"https://audioboom.com/channels/5081029.rss", "trackViewUrl":"https://podcasts.apple.com/us/podcast/joe-rogan-experience-review-podcast/id1396757201?uo=4", "artworkUrl30":"https://is4-ssl.mzstatic.com/image/thumb/Podcasts122/v4/e0/73/74/e07374ec-a986-feed-027c-5af718895a31/mza_4297001239469570007.jpg/30x30bb.jpg", "artworkUrl60":"https://is4-ssl.mzstatic.com/image/thumb/Podcasts122/v4/e0/73/74/e07374ec-a986-feed-027c-5af718895a31/mza_4297001239469570007